In [1]:
from torch import Tensor
from torch.nn import Module
from torch.nn import Linear 
from torch.nn import Identity

class Neuron(Module):
    """
    Represents a single neuron capable of aggregating inputs and applying an activation function.
    A neuron is capable of learning linear relationships in data.

    Args:
        input_features (int): Number of input features.
        activation (Module, optional): Activation function to apply after the linear transformation. 
                                       Defaults to Identity (no activation).
    """
    def __init__(self, connections: int, activation: Module | None = None):
        super().__init__()
        self.transformation = Linear(connections, 1) 
        self.activation = activation or Identity()

    def forward(self, signals: Tensor):
        return self.activation(self.transformation(signals))

In [2]:
from torch.optim import Optimizer

class GD(Optimizer):
    def __init__(self, params, lr: float):
        if lr < 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        defaults = dict(lr=lr)
        super().__init__(params, defaults)


    def step(self): 
        """Performs a single optimization step (no closure support)."""
        for group in self.param_groups:
            lr = group['lr']
            
            for param in group['params']:
                if param.grad is None:
                    continue
                 
                param.data -= lr * param.grad.data

In [3]:
from torch import Tensor
from torch import from_numpy
from torch.utils.data import Dataset
from numpy import float32
from sklearn.datasets import make_regression

class Hyperplane(Dataset):
    """
    Represents a dataset of points in a hyperplane.

    Args:
        dimension (int): Number of dimensions of the hyperplane.<
        samples (int): Number of samples to generate.
        noise (float): Standard deviation of the Gaussian noise added to the targets.
        random_state (int, optional): Random seed for reproducibility. Defaults to 42.
    """
    def __init__(self, dimension: int, samples: int, noise: float, random_state: int = 42): 
        self.features, self.targets = make_regression(samples, dimension, noise=noise, random_state=random_state)  
        self.features, self.targets = from_numpy(self.features.astype(float32)), from_numpy(self.targets.astype(float32).reshape(-1, 1))

    def __len__(self) :
        return len(self.features)
    
    def __getitem__(self, index: int) -> tuple[Tensor, Tensor]:
        return self.features[index], self.targets[index]

In [4]:
from torchsystem.services import Consumer, event
from torchsystem.depends import Depends

consumer = Consumer()

def train(model: Neuron, criterion: Module, optimizer: Optimizer, dataset: Hyperplane):
    model.train()
    optimizer.zero_grad()
    outputs = model(dataset.features)
    loss = criterion(outputs, dataset.targets)
    loss.backward()
    optimizer.step()
    consumer.consume(Trained(model, loss))

@event
class Trained:
    model: Neuron
    loss: Tensor

def store() -> dict[str, list]:
    ...

@consumer.handler
def handle_history(event: Trained, store: dict[str, list] = Depends(store)):
    store['loss'].append(event.loss.item())
    store['bias'].append(event.model.transformation.bias.item())
    store['weight'].append(event.model.transformation.weight.item())

In [5]:
from torch import Tensor
from torch import linspace
from plotly.graph_objects import Figure, Scatter  

def plot_linear_samples(features: Tensor, targets: Tensor, figure: Figure | None = None) -> Figure:
    figure = figure or Figure()
    figure.add_trace(
        Scatter(
            x=features.flatten(),
            y=targets.flatten(),
            mode='markers',
            name='Samples',
            marker=dict(
                size=8,
                color='blue',
                opacity=0.7
            )
        )
    )

    figure.update_layout(
        title='Linear Samples',
        xaxis_title='X',
        yaxis_title='Y',
        showlegend=True
    )
    return figure


def plot_line(slope: float, intercept: float, figure: Figure | None = None) -> Figure:
    figure = figure or Figure()
    x = linspace(-4, 4, 100) 
    y = slope * x + intercept
    figure.add_trace(
        Scatter(
            x=x,
            y=y,
            mode='lines',
            name=f'y = {slope:.2f}x + {intercept:.2f}',
            line=dict(
                color='red',
                width=3
            )
        )
    )
     
    figure.add_trace(
        Scatter(
            x=[0],
            y=[intercept],
            mode='markers',
            name=f'Intercept (0, {intercept:.2f})',
            marker=dict(
                color='green',
                size=10
            )
        )
    )
    
    figure.update_layout(
        title=f'Linear Function: y = {slope:.2f}x + {intercept:.2f}',
        xaxis_title='X',
        yaxis_title='Y',
        showlegend=True
    )
    
    return figure

In [6]:
from torch import meshgrid
from torch import linspace
from torch import mean
from torch import zeros

from plotly.graph_objects import Figure
from plotly.graph_objects import Surface

def plot_loss_surface(features: Tensor, targets: Tensor, figure: Figure | None = None):
    weights = linspace(-10, 60, 100) 
    biases = linspace(-50, 50, 100) 
    losses = zeros((len(weights), len(biases)))
    for i, weight in enumerate(weights):
        for j, bias in enumerate(biases):
            predictions = features * weight + bias
            losses[i, j] = mean((predictions - targets)**2)
    
    
    weights_grid, biases_grid = meshgrid(weights, biases, indexing='ij')

    figure = figure or Figure()
    figure.add_trace(Surface(
        x=weights_grid,
        y=biases_grid,
        z=losses,
        colorscale='Viridis',
        name='Loss surface'
    ))
     
    figure.update_layout(
        title='Loss Surface',
        scene=dict(
            xaxis_title='Weights',
            yaxis_title='Biases',
            zaxis_title='Loss',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=0.8)  # Adjust camera view
            )
        ),
        margin=dict(l=0, r=0, b=0, t=30)
    )
    return figure    

In [7]:
from plotly.graph_objects import Scatter3d

def plot_training_history(history: dict[str, list], figure: Figure | None = None):
    figure = figure or Figure()
    figure.add_trace(Scatter3d(
        x=history['weight'],
        y=history['bias'],
        z=history['loss'],
        mode='lines+markers',
        marker=dict(
            size=4,
            color=history['loss'],  
            colorscale='reds', 
        ),
        line=dict(
            color='red',
            width=2
        ),
        name='Training path'
    ))
     
    return figure

In [8]:
from torch.nn import MSELoss

model = Neuron(connections=1)
criterion = MSELoss()
optimizer = GD(model.parameters(), lr=0.01)
dataset = Hyperplane(1, 100, noise=20)  
history = {
    'loss': [],
    'bias': [],
    'weight': []
}
consumer.override(store, lambda: history)

for epoch in range(1000):
    train(model, criterion, optimizer, dataset)

In [9]:
figure = plot_linear_samples(dataset.features, dataset.targets)
figure = plot_line(model.transformation.weight.item(), model.transformation.bias.item(), figure)
figure.show()


figure = plot_loss_surface(dataset.features, dataset.targets) 
figure = plot_training_history(history, figure) 
figure.show()

In [10]:
from torch.linalg import pinv
from torch import cat, ones
 
dataset = Hyperplane(1, 100, noise=20)
model = Neuron(1) 
features  = cat([dataset.features , ones(dataset.features .shape[0], 1)], dim=1)
parameters = pinv(features) @ dataset.targets 
model.transformation.weight.data = parameters[0] 
model.transformation.bias.data = parameters[1]  
figure = plot_linear_samples(dataset.features, dataset.targets)
figure = plot_line(model.transformation.weight.item(), model.transformation.bias.item(), figure)
figure.show()